#  Analyste Financier avec LLaMa - 3

---
Dans ce notebook nous allons:
1. Affiner localement un modèle LLaMa 3 à l’aide de données de questions-réponses contextuelles issues de formulaires 10-K, en utilisant un apprentissage supervisé et une adaptation à bas rang (LoRA).

2. Mettre en place un pipeline de données pour récupérer les derniers rapports 10-K auprès de la SEC.

3. Utiliser des embeddings locaux et une base vectorielle en mémoire pour créer une fonction de recherche (retrieval).

4. Combiner le tout afin de construire un agent RAG (Retrieval-Augmented Generation) destiné à l’analyse financière.

Par Yvan-Manuel BALEGUEL, inspiré de [Data Camp Tutorial] (https://www.datacamp.com/tutorial/llama3-fine-tuning-locally)

---

### Installation des dépendances et configuration des clés API

Dans cette section, nous allons :

1. Installer toutes les bibliothèques nécessaires à l'entraînement du modèle, à la récupération des données financières et à la création de la base de données vectorielle locale.

2. Ajouter les clés d'authentification API pour Hugging Face (accès aux modèles LLaMa) et, si besoin, pour l'API SEC (accès aux rapports financiers 10-K).



In [ ]:
%%capture
# Installe Unsloth, Xformers (Flash Attention) et autres packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install sec_api
!pip install -U langchain
!pip install -U langchain-community
!pip install -U sentence-transformers
!pip install -U faiss-gpu
!pip install faiss-gpu
!pip install python-dotenv

Le Token HF et l'api SEC ci-dessosus sont personnels et à ne pas pertager.
Baleguel Yvan-Manuel

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
# HuggingFace token, nécessaire pour accéder aux modèles  (comme LLaMa 3 8B Instruct)
hf_token = os.getenv("HF_TOKEN")
# SEC-API Key
sec_api_key = os.getenv("SEC_API_KEY")

In [28]:
# Packages pour Fine Tuning
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Packages pour Pipeline & RAG
from sec_api import ExtractorApi, QueryApi
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

---
## **Partie 1: Fine Tuning LLaMa 3 avec Unsloth**

Nous allons utiliser le **GPU intégré de Colab** pour effectuer tout le processus de **fine-tuning**, en utilisant la bibliothèque [**Unsloth**](https://github.com/unslothai/unsloth).

Une grande partie du code ci-dessous est **adaptée à partir de la [documentation officielle d’Unsloth](https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing#scrollTo=AEEcJ4qfC7Lp)**.



### **Initialisation du modèle pré-entraîné et du tokenizer**

Dans cet exemple, nous utiliserons le modèle [**LLaMa 3 8B Instruct de Meta**](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct).  
**REMARQUE** : Il s'agit d’un modèle restreint ("gated model") — vous devez **demander un accès sur Hugging Face** et **fournir votre token HF** dans l’étape ci-dessous.


In [4]:
# Charger le modèle et tokenizer du pre-trained FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    # Sélectionner le pre-trained model à utiliser
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct",
    # Spécifier le nombre de Tokens max que le modèle peut process en un seul forward
    max_seq_length = 2048,
    # Type de données utilisé pour le modèle. None signifie que le type est détecté automatiquement selon le matériel disponible. Float16 est recommandé pour certains GPU spécifiques, comme le Tesla T4.
    dtype = None,
    # Activer la quantification en 4 bits. En quantifiant les poids du modèle en 4 bits au lieu des 16 ou 32 bits habituels,la mémoire nécessaire pour stocker ces poids est fortement réduite. Cela permet de faire tourner des modèles plus gros sur du matériel avec une mémoire limitée.
    load_in_4bit = True,
    # Jeton d'accès pour les modèles restreints ("gated models"), requis pour s'authentifier et utiliser des modèles comme Meta-Llama-2-7b-hf.
    token = hf_token,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

**Ajout des adaptateurs LoRA pour un fine-tuning efficace en paramètres**

LoRA, ou **Low-Rank Adaptation**, est une technique utilisée en apprentissage automatique pour **affiner plus efficacement les grands modèles**.  
Elle consiste à ajouter un petit ensemble de paramètres supplémentaires au modèle existant, **sans avoir à réentraîner tous les paramètres** d’origine.

Cela rend le processus de fine-tuning **plus rapide** et **moins gourmand en ressources**.  
En résumé, LoRA permet d’adapter un modèle pré-entraîné à des tâches ou jeux de données spécifiques, **sans nécessiter une puissance de calcul ou une mémoire importante**.


In [5]:
# Apply LoRA (Low-Rank Adaptation) adapters to the model for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    # Appliquer les adaptateurs LoRA (Low-Rank Adaptation) au modèle pour un fine-tuning efficace avec un nombre réduit de paramètres à entraîner.
    r = 16,
    # Spécifier les couches du modèle auxquelles les adaptateurs LoRA doivent être appliqués.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    # Facteur d’échelle pour LoRA.Contrôle l’importance de l’adaptation. En général, c’est un petit entier positif.
    lora_alpha = 16,
    # Taux de dropout (abandon) pour LoRA.Une valeur de 0 signifie aucun dropout, ce qui est optimal pour les performances.
    lora_dropout = 0,
    # Gestion du biais dans LoRA. Le paramètre "none" est optimisé pour les performances, mais d'autres options peuvent être utilisées selon les besoins.
    bias = "none",
    # Active le gradient checkpointing pour économiser de la mémoire pendant l'entraînement. L'option "unsloth" est optimisée pour les contextes très longs.
    use_gradient_checkpointing = "unsloth",
    # Graine pour la génération de nombres aléatoires, afin de garantir la reproductibilité des résultats.
    random_state = 3407,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


1. **r** : Le **rang** de la matrice d’adaptation à bas rang (Low-Rank Adaptation).  
   Il détermine la capacité de l’adaptateur à capturer des informations supplémentaires.  
   Un rang plus élevé permet de modéliser des motifs plus complexes, mais augmente également le coût de calcul.

2. **target_modules** : Liste des **couches du modèle** auxquelles les adaptateurs LoRA doivent être appliqués.  
   Il s’agit en général des projections internes aux couches transformeurs (transformer layers), telles que :

   - **q_proj** : Projette les caractéristiques d’entrée vers les vecteurs de requêtes (queries) pour le mécanisme d’attention.
   - **k_proj** : Projette les caractéristiques d’entrée vers les vecteurs de clés (keys).
   - **v_proj** : Projette les caractéristiques d’entrée vers les vecteurs de valeurs (values).
   - **o_proj** : Projette la sortie du mécanisme d’attention vers la couche suivante.
   - **gate_proj** : Applique des mécanismes de « gating » pour réguler le flux d’information.
   - **up_proj** : Projette les caractéristiques dans un espace de plus haute dimension (feed-forward).
   - **down_proj** : Projette les caractéristiques dans un espace de plus basse dimension.

   Ces couches sont essentielles au bon fonctionnement des modèles basés sur les transformeurs, notamment pour les calculs d’attention et les transformations dans les réseaux feed-forward.

3. **lora_alpha** : Facteur d’échelle pour l’adaptateur LoRA.  
   Il contrôle l’impact de l’adaptateur sur les sorties du modèle.  
   En général, on utilise un petit entier positif.

4. **lora_dropout** : Taux de dropout appliqué aux adaptateurs LoRA.  
   Le dropout aide à régulariser l’apprentissage.  
   Une valeur de `0` signifie **aucun dropout**, ce qui est souvent optimal pour les performances.

5. **bias** : Spécifie la gestion des biais dans les adaptateurs LoRA.  
   La valeur `"none"` désactive les biais pour optimiser les performances,  
   mais d’autres options sont possibles selon le cas d’usage.

6. **use_gradient_checkpointing** : Active le **gradient checkpointing**,  
   ce qui permet d’économiser de la mémoire pendant l’entraînement en **ne stockant pas tous les états intermédiaires**.  
   L’option `"unsloth"` est optimisée pour les contextes très longs, mais on peut aussi la remplacer par `True`.

7. **random_state** : Graine du générateur de nombres aléatoires,  
   utilisée pour assurer la **reproductibilité** des résultats entre différentes exécutions du code.


### **Préparation du jeu de données pour le fine-tuning**

Nous allons utiliser un jeu de données Hugging Face de **questions-réponses financières extraites de rapports 10-K**, mis à disposition par l’utilisateur [**Virat Singh**](https://github.com/virattt) :  
https://huggingface.co/datasets/virattt/llama-3-8b-financialQA

Le code ci-dessous formate les entrées selon le **prompt d'entraînement défini plus tôt**, en veillant à bien ajouter les **tokens spéciaux** nécessaires.  
Dans notre cas, le **token de fin de phrase** est `<|eot_id|>`.

La liste complète des tokens spéciaux pour LLaMa 3 est disponible [ici](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/)


In [6]:
# Définir le prompt exact attendu
ft_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}

### Context:
{}

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

# Récupérer la fin de phrase Token Spécial
EOS_TOKEN = tokenizer.eos_token # Doit ajouter EOS_TOKEN

# Fonction permettant de formater le prompt ci-dessus à partir des informations du jeu de données Financial QA.

def formatting_prompts_func(examples):
    questions = examples["question"]
    contexts       = examples["context"]
    responses      = examples["answer"]
    texts = []
    for question, context, response in zip(questions, contexts, responses):
        # Il est impératif d’ajouter le EOS_TOKEN, sinon la génération risque de ne jamais s’arrêter !

        text = ft_prompt.format(question, context, response) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("virattt/llama-3-8b-financialQA", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'answer': 'NVIDIA initially focused on PC graphics.',
 'context': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'ticker': 'NVDA',
 'filing': '2023_10K',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nBelow is a user question, paired with retrieved context. Write a response that appropriately answers the question,\ninclude specific details in your response. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n### Question:\nWhat area did NVIDIA initially focus on before expanding to other computationally intensive fields?\n\n### Context:\nSince our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.\n\n<|eot_id|>\n\n### Response: <|start_header_id|>assistant<|end_header_id|>\nNVIDIA initially

### **Définition des arguments du Trainer**

Nous allons configurer et utiliser le [**Supervised Fine-Tuning Trainer**](https://huggingface.co/docs/trl/sft_trainer) de la librairie **HuggingFace TRL (Transformer Reinforcement Learning)**.

Le **fine-tuning supervisé** est un processus en apprentissage automatique où un modèle pré-entraîné est ensuite entraîné sur un **jeu de données spécifique avec des exemples annotés**.  
Pendant ce processus, le modèle apprend à faire des prédictions ou des classifications en se basant sur ces exemples, améliorant ainsi sa performance sur la tâche ciblée.

Cette technique permet de tirer parti des **connaissances générales** acquises lors de la phase de pré-entraînement, tout en adaptant le modèle à un **contexte ou domaine précis**.

Le fine-tuning supervisé est couramment utilisé pour personnaliser un modèle sur des applications spécifiques telles que :

- l’analyse de sentiments,  
- la reconnaissance d’objets,  
- ou la traduction automatique,  

en s’appuyant sur des données annotées selon la tâche visée.


In [10]:
trainer = SFTTrainer(
    # Le modèle qui doit être fine-tuned
    model = model,
    # Le Tokenizer associé au modèle
    tokenizer = tokenizer,
    # Le dataset urilisé pour training
    train_dataset = dataset,
    # La partie du dataset avec le texte
    dataset_text_field = "text",
    # Longueur maximale des séquences pour les données d'entraînement
    max_seq_length = 2048,
    # Nombre de processus à utiliser pour le chargement des données
    dataset_num_proc = 2,
    # Utiliser ou non le "sequence packing", ce qui peut accélérer l'entraînement pour les séquences courtes
    packing = False,
    args = TrainingArguments(
    # Taille de batch par appareil pendant l'entraînement
    per_device_train_batch_size = 2,
    # Nombre d'étapes d'accumulation de gradients avant la mise à jour des paramètres du modèle
    gradient_accumulation_steps = 4,
    # Nombre d'étapes de "warmup" pour le scheduler du taux d'apprentissage
    warmup_steps = 5,
    # Nombre total d'étapes d'entraînement
    max_steps = 60,
    # Nombre d'époques d'entraînement — peut être utilisé à la place de max_steps ; ici, le dataset permettrait ~900 étapes
    # num_train_epochs = 1,
    # Taux d'apprentissage de l'optimiseur
    learning_rate = 2e-4,
    # Utiliser la précision en virgule flottante 16 bits si bfloat16 n'est pas supporté
    fp16 = not is_bfloat16_supported(),
    # Utiliser la précision bfloat16 si elle est supportée
    bf16 = is_bfloat16_supported(),
    # Nombre d'étapes entre chaque enregistrement de logs
    logging_steps = 1,
    # Optimiseur à utiliser (ici, AdamW en précision 8 bits)
    optim = "adamw_8bit",
    # Décroissance de poids à appliquer aux paramètres du modèle (régularisation)
    weight_decay = 0.01,
    # Type de scheduler utilisé pour le taux d'apprentissage
    lr_scheduler_type = "linear",
    # Graine pour la génération de nombres aléatoires afin d'assurer la reproductibilité
    seed = 3407,
    # Dossier de sortie pour sauvegarder les modèles et les logs
    output_dir = "outputs",
  ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

1. **model** : Le modèle à affiner.  
   Il s'agit du **modèle pré-entraîné** qui sera adapté au jeu de données d'entraînement spécifique.

2. **tokenizer** : Le tokenizer associé au modèle.  
   Il transforme les textes en **tokens** exploitables par le modèle.

3. **train_dataset** : Le jeu de données utilisé pour l'entraînement.  
   C'est l'ensemble d'exemples annotés à partir duquel le modèle va apprendre.

4. **dataset_text_field** : Le champ contenant les textes dans le dataset.  
   Il indique **quelle colonne** contient les données textuelles à utiliser pour l'entraînement.

5. **max_seq_length** : Longueur maximale des séquences.  
   Elle limite le nombre de tokens par entrée pour respecter la capacité du modèle.

6. **dataset_num_proc** : Nombre de processus utilisés pour charger les données.  
   Cela permet d'accélérer le chargement des données par **traitement parallèle**.

7. **packing** : Booléen indiquant si le **regroupement de séquences** est activé.  
   Le "packing" permet de **regrouper plusieurs courtes séquences** dans un même lot, ce qui accélère l'entraînement.

8. **args** : Ensemble d’**arguments de configuration de l'entraînement** (hyperparamètres) :

   - **per_device_train_batch_size** : Taille du batch par appareil pendant l'entraînement.  
     Cela détermine combien d'exemples sont traités à chaque itération.

   - **gradient_accumulation_steps** : Nombre d'étapes d'accumulation de gradients avant mise à jour des poids.  
     Cela permet de simuler un batch plus grand sans dépasser la mémoire disponible.

   - **warmup_steps** : Nombre d’étapes de "chauffe" pour le scheduler de taux d’apprentissage.  
     Le learning rate augmente progressivement durant ces étapes initiales.

   - **max_steps** : Nombre total d'étapes d’entraînement.  
     Définit combien de batchs seront utilisés au total pour l’apprentissage.

   - **num_train_epochs** : Nombre d’**époques d’entraînement** (commenté dans l’exemple).  
     Définit combien de fois l’ensemble du dataset sera parcouru par le modèle.

   - **learning_rate** : Taux d’apprentissage pour l’optimiseur.  
     Contrôle l’intensité des ajustements de poids à chaque mise à jour.

   - **fp16** : Booléen pour activer l’entraînement en virgule flottante 16 bits si bfloat16 n’est pas supporté.  
     Permet de réduire la mémoire utilisée et accélérer l’entraînement.

   - **bf16** : Booléen pour activer le format **bfloat16**, plus stable que fp16 si disponible.

   - **logging_steps** : Fréquence (en nombre d’étapes) à laquelle les logs d'entraînement sont enregistrés.

   - **optim** : Optimiseur utilisé.  
     Ici, **AdamW en 8 bits**, qui améliore l'efficacité mémoire pour les grands modèles.

   - **weight_decay** : Taux de **pénalisation des grands poids** (régularisation).  
     Réduit le risque de surapprentissage (overfitting).

   - **lr_scheduler_type** : Type de scheduler utilisé pour le learning rate.  
     Contrôle l’évolution du taux d’apprentissage au fil du temps.

   - **seed** : Graine utilisée pour la génération aléatoire, garantissant la **reproductibilité** des résultats.

   - **output_dir** : Dossier de sortie où seront **enregistrés le modèle entraîné et les logs** d'entraînement.


# **Prêt pour l'entrainement!**

In [15]:
import wandb
wandb.init(project="mon_projet_llama3")
trainer_stats = trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yvanmanuelbbm (yvanmanuelbbm-centralesup-lec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,4.553500
2,4.016900
3,4.007900
4,3.704900
5,2.684600
6,2.541600
7,2.093900
8,2.158200
9,2.010900
10,1.589800


---
### **Sauvegarde locale de votre modèle fine-tuné**

Commencez par cliquer sur l’onglet **Fichiers**, puis **montez Google Drive** pour y accéder depuis Colab.  
Créez ensuite un dossier dans votre Drive et remplacez le chemin d’enregistrement ci-dessous par le vôtre.


In [16]:
model.save_pretrained("/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60") # Sauvegarde locale
tokenizer.save_pretrained("/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60")

('/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60/tokenizer_config.json',
 '/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60/special_tokens_map.json',
 '/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60/tokenizer.json')

### **Fonction pour recharger votre modèle fine-tuné plus tard**

Pour éviter de devoir réentraîner le modèle à chaque fois, il suffit de **remplacer `False` par `True`** et d’exécuter ce bloc.  


In [17]:
# Redéfinition du prompt si le modèle est importé sans passer par la phase d'entraînement
ft_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}

### Context:
{}

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

if False: # switch à true pour charger le back up du modèle
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/projet_ia_centralesup_finance/projet_ia_centralesup_finance_step60", # Chemin vers modèle sauvegardé
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

### **Mise en place des fonctions pour exécuter l’inférence**

L’**inférence** désigne le processus d’utilisation d’un modèle de machine learning entraîné pour **faire des prédictions ou générer du contenu** à partir de nouvelles données, jamais vues auparavant.  
Elle consiste à **fournir une entrée au modèle**, qui retourne ensuite une **prédiction, une classification ou un texte généré**, selon la tâche pour laquelle il a été conçu.

C’est la **phase d’application du modèle**, par opposition à la phase d’entraînement.


In [22]:
# Fonction principale d'inférence
def inference(question, context):
  inputs = tokenizer(
  [
      ft_prompt.format(
          question,
          context,
          "", # output - à laisser vide pour géneration!
      )
  ], return_tensors = "pt").to("cuda")

  # Génère des tokens à partir du prompt d’entrée en utilisant le modèle,
  # avec un maximum de 64 nouveaux tokens générés.
  # Le paramètre `use_cache` permet d’accélérer la génération en réutilisant les calculs précédents.
  # Le `pad_token_id` est défini sur le token de fin (EOS) pour gérer correctement le padding.

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id=tokenizer.eos_token_id)
  response = tokenizer.batch_decode(outputs) # Décode le Token en mots anglais
  return response

In [21]:
# Fonction permettant d'extraire uniquement la réponse générée par le modèle à partir de la sortie complète du prompt.

def extract_response(text):
    text = text[0]
    start_token = "### Response: <|start_header_id|>assistant<|end_header_id|>"
    end_token = "<|eot_id|>"

    start_index = text.find(start_token) + len(start_token)
    end_index = text.find(end_token, start_index)

    if start_index == -1 or end_index == -1:
        return None

    return text[start_index:end_index].strip()

In [23]:
# Test!
context = "The increase in research and development expense for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure."
question = "What were the primary drivers of the notable increase in research and development expenses for fiscal year 2023?"

resp = inference(question, context)
parsed_response = extract_response(resp)
print(parsed_response)

The notable increase in research and development expenses for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure.


In [24]:
print(resp)

['<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\nBelow is a user question, paired with retrieved context. Write a response that appropriately answers the question,\ninclude specific details in your response. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n### Question:\nWhat were the primary drivers of the notable increase in research and development expenses for fiscal year 2023?\n\n### Context:\nThe increase in research and development expense for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure.\n\n<|eot_id|>\n\n### Response: <|start_header_id|>assistant<|end_header_id|>\nThe notable increase in research and development expenses for fiscal year 2023 was primarily driven by increased compensation, employee growth, engineering development costs, and data center infrastructure.<|eot_id|>']


---
# **Partie 2 : Mise en place du pipeline de données SEC 10-K et de la fonctionnalité de recherche (retrieval)**

Maintenant que nous avons notre **modèle de langage fine-tuné**, nos **fonctions d’inférence** et un **format de prompt bien défini**,  
nous allons mettre en place un pipeline **RAG** (Retrieval-Augmented Generation) pour injecter automatiquement le **contexte pertinent** dans chaque génération.

Voici le flux que nous allons construire :

**Question de l’utilisateur** → **Récupération du contexte depuis le 10-K** → **Le LLM répond en s’appuyant sur ce contexte**

Pour cela, nous devons être capables de :

1. Récupérer des informations précises depuis les rapports 10-K  
2. Parser et découper le texte de ces documents  
3. Vectoriser et transformer ces segments en **embeddings**, stockés dans une base de données vectorielle  
4. Mettre en place un **retriever** qui effectue une recherche sémantique à partir des questions utilisateur, afin de retourner le contexte le plus pertinent

Un **Formulaire 10-K** est un rapport annuel obligatoire déposé auprès de la **U.S. Securities and Exchange Commission (SEC)**.  
Il fournit un résumé détaillé de la performance financière d’une entreprise sur l’année écoulée.


### **Fonction de récupération des rapports 10-K**

Pour simplifier cette étape, nous utilisons l’API de la SEC : https://sec-api.io/.  
L’inscription est gratuite et permet d’effectuer **100 appels par jour**.  
Chaque chargement de symboles boursiers (tickers) consomme environ **3 appels API**.

Dans ce projet, nous allons nous concentrer uniquement sur les sections suivantes des rapports 10-K :

- **Section 1A** : Facteurs de risque (*Risk Factors*)  
- **Section 7** : Analyse de la situation financière et des résultats d’exploitation (*Management's Discussion and Analysis of Financial Condition and Results of Operations*)


In [25]:
# Extraction des fillings
def get_filings(ticker):
    global sec_api_key

    # cherche les Filings récents avec QueryAPI
    queryApi = QueryApi(api_key=sec_api_key)
    query = {
      "query": f"ticker:{ticker} AND formType:\"10-K\"",
      "from": "0",
      "size": "1",
      "sort": [{ "filedAt": { "order": "desc" } }]
    }
    filings = queryApi.get_filings(query)

    # 10-K URL
    filing_url = filings["filings"][0]["linkToFilingDetails"]

    # Extrait le Texte avec ExtractorAPI
    extractorApi = ExtractorApi(api_key=sec_api_key)
    onea_text = extractorApi.get_section(filing_url, "1A", "text") # Section 1A - Risk Factors
    seven_text = extractorApi.get_section(filing_url, "7", "text") # Section 7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations

    # Joindre les Textes
    combined_text = onea_text + "\n\n" + seven_text

    return combined_text

### **Configuration locale des embeddings**

Dans l’esprit d’une approche **locale et fine-tunée**, nous utiliserons un modèle open source d’embedding développé par la **Beijing Academy of Artificial Intelligence** :  
👉 [**Large English Embedding Model**](https://huggingface.co/BAAI/bge-large-en-v1.5)  
Plus d’informations disponibles dans leur [dépôt GitHub](https://github.com/FlagOpen/FlagEmbedding)

---

**Les embeddings** sont des représentations numériques de données, utilisées pour convertir des informations complexes et de haute dimension en un espace vectoriel de plus faible dimension.  
Dans le domaine du **traitement du langage naturel (NLP)**, les embeddings servent à représenter des mots, phrases ou documents sous forme de **vecteurs de nombres réels**.

Ces vecteurs capturent les **relations sémantiques** : autrement dit, des mots ou phrases ayant un sens proche auront des vecteurs proches dans l’espace vectoriel.

---

**Les modèles d’embedding** sont des modèles d’apprentissage automatique conçus pour apprendre ces représentations.  
Ils sont entraînés à encoder divers types de données tout en conservant leurs caractéristiques et relations essentielles.

Par exemple, en NLP, des modèles comme **Word2Vec**, **GloVe** ou **BERT** sont entraînés sur de larges corpus pour produire des embeddings exploitables dans de nombreuses tâches :  
- classification de texte,  
- analyse de sentiment,  
- traduction automatique, etc.

Dans notre cas, les embeddings seront utilisés pour **mesurer la similarité sémantique** entre une question et des documents financiers.


In [26]:
# Chemin modèle HF
modelPath = "BAAI/bge-large-en-v1.5"
# Crée un dictionnaire avec les options de configuration du modèle,en précisant l’utilisation de CUDA pour optimiser l’exécution sur GPU.
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': True}

# Initialise une instance des embeddings HuggingFace de LangChain en utilisant les paramètres spécifiés précédemment.
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # chemin vers le pre-trained model
    model_kwargs=model_kwargs, # Pass les options de config du modèle
    encode_kwargs=encode_kwargs # Pass les encoding options
)

<ipython-input-26-fb49199d37cb>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### **Traitement et définition de la base de données vectorielle**

Dans cette partie, nous allons utiliser les données récupérées via les fonctions de l’API SEC, puis les traiter en suivant trois étapes :

1. **Découpage du texte**  
2. **Vectorisation**  
3. **Mise en place de la fonction de recherche (retrieval)**

---

#### 🔹 **Découpage du texte (Text Splitting)**  
Le découpage consiste à diviser un document volumineux (rapport financier, document juridique, etc.) en **segments plus petits et gérables**.  
Cela permet de mieux traiter, analyser et indexer le contenu à l’aide de modèles d’IA ou de bases de données.

---

#### 🔹 **Bases de données vectorielles (Vector Databases)**  
Ces bases stockent les données sous forme de **vecteurs numériques** (embeddings), qui capturent le **sens sémantique** du texte, d’une image ou d’autres types de données.  
Elles permettent ensuite des **recherches par similarité** très efficaces.

Dans ce projet, nous utilisons **[FAISS](https://ai.meta.com/tools/faiss/)**, la librairie de recherche vectorielle développée par **Facebook AI**.  
C’est une solution légère, **entièrement en mémoire** (pas besoin de sauvegarde disque), parfaitement adaptée à notre cas d’usage, même si moins puissante que d’autres bases vectorielles industrielles.

---

#### 🔁 **Comment les documents découpés et les embeddings sont utilisés ensemble :**

1. **Embeddings** : Chaque segment de texte découpé est transformé en **vecteur numérique** à l’aide d’un modèle d’embedding.  
   Ces vecteurs représentent le sens sémantique du texte.

2. **Stockage** : La base vectorielle conserve les embeddings **ainsi qu’un lien vers le texte d’origine**.

3. **Indexation** : Les vecteurs sont indexés pour permettre une **recherche rapide et efficace** par similarité.

4. **Utilisation** : Lors d’une requête, la base vectorielle retrouve les **vecteurs les plus proches** du vecteur de la question, et renvoie les **passages de texte pertinents** à utiliser comme contexte.


In [34]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.8 MB/s eta 0:00:00


In [36]:
# Prompt input par l'utilisateur pour le stock ticker à analyser
ticker = input("What Ticker Would you Like to Analyze? ex. AAPL: ")

print("-----")
print("Getting Filing Data")
# récupérer les fillings pour le ticker
filing_data = get_filings(ticker)

print("-----")
print("Initializing Vector Database")
# Initialise un découpeur de texte pour diviser le contenu en segments
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,         # Taille maximale de chaque segment
    chunk_overlap = 500,       # Nombre de caractères de chevauchement entre segments
    length_function = len,     # Fonction utilisée pour mesurer la longueur des segments
    is_separator_regex = False # Le séparateur n'est pas une expression régulière
)
# Découpe le contenu du rapport en segments exploitables
split_data = text_splitter.create_documents([filing_data])

# Crée une base vectorielle FAISS à partir des segments et des embeddings
db = FAISS.from_documents(split_data, embeddings)

# Crée un objet de récupération pour effectuer des recherches dans la base vectorielle
retriever = db.as_retriever()

print("-----")
print("Filing Initialized")


What Ticker Would you Like to Analyze? ex. AAPL: AAPL
-----
Getting Filing Data
-----
Initializing Vector Database
-----
Filing Initialized



### **Recherche (Retrieval)**

**Description :**  
La recherche (retrieval) consiste à **interroger une base de données vectorielle** afin de retrouver les **segments de texte les plus pertinents** par rapport à une requête donnée.  
Cela implique de parcourir les embeddings indexés pour identifier ceux qui sont les plus proches de l’embedding de la question.

---

**Fonctionnement :**

1. **Embedding de la requête** :  
   Lorsqu’une question est posée, elle est d’abord **convertie en vecteur (embedding)** à l’aide du **même modèle d’embedding** que celui utilisé pour les documents.

2. **Recherche par similarité** :  
   Le système effectue une recherche des vecteurs les plus proches dans la base vectorielle.  
   La **similarité** est généralement mesurée à l’aide de **distances cosinus** ou **euclidiennes**.

3. **Récupération des documents** :  
   Le système identifie les **segments de texte originaux** associés aux vecteurs similaires retrouvés.

4. **Assemblage du contexte** :  
   Les segments de texte sélectionnés sont **assemblés** pour fournir un **contexte cohérent** qui sera injecté dans le prompt du modèle.

---

Dans la fonction ci-dessous, la requête sert à **invoquer le retriever**,  
qui renvoie une liste de documents.  
Le contenu de ces documents est ensuite extrait et retourné en tant que **contexte** pour le modèle.


In [37]:
def retrieve_context(query):
    global retriever
    retrieved_docs = retriever.invoke(query)
    context = []
    for doc in retrieved_docs:
        context.append(doc.page_content)
    return context

In [38]:
context = retrieve_context("How have currency fluctuations impacted the company's net sales and gross margins?")
print(context)

['The weakening of foreign currencies relative to the U.S. dollar adversely affects the U.S. dollar value of the Company&#8217;s foreign currency&#8211;denominated sales and earnings, and generally leads the Company to raise international pricing, potentially reducing demand for the Company&#8217;s products. In some circumstances, for competitive or other reasons, the Company may decide not to raise international pricing to offset the U.S. dollar&#8217;s strengthening, which would adversely affect the U.S. dollar value of the gross margins the Company earns on foreign currency&#8211;denominated sales.', 'The Company&#8217;s profit margins vary across its products, services, geographic segments and distribution channels. For example, the gross margins on the Company&#8217;s products and services vary significantly and can change over time. The Company&#8217;s gross margins are subject to volatility and downward pressure due to a variety of factors, including: continued industry-wide glo

---
# **Script principal : tout assembler !**

Nous allons maintenant **regrouper toutes les étapes précédentes** dans une boucle `while` très simple.  
Cette boucle va :

1. Prendre une **question de l’utilisateur**  
2. **Récupérer le contexte pertinent** depuis la base vectorielle alimentée par le rapport 10-K de l’entreprise concernée  
3. Lancer l’**inférence avec notre modèle fine-tuné** pour générer une réponse

Essayez par vous-même !


In [39]:
while True:
  question = input(f"What would you like to know about {ticker}'s form 10-K? ")
  if question == "x":
    break
  else:
    context = retrieve_context(question) # Context Retrieval
    resp = inference(question, context) # Running Inference
    parsed_response = extract_response(resp) # Parsing Response
    print(f"L3 Agent: {parsed_response}")
    print("-----\n")


What would you like to know about AAPL's form 10-K? Where is outsourcing located currently?
L3 Agent: The outsourcing partners are located primarily in China mainland, India, Japan, South Korea, Taiwan, and Vietnam.
-----

What would you like to know about AAPL's form 10-K? Does the US dollar weakening help or hurt the company?
L3 Agent: The weakening of the US dollar relative to other currencies generally negatively affects the company's sales and earnings due to reduced demand and lower gross margins.
-----



KeyboardInterrupt: Interrupted by user

What region contributes most to international sales?  
Where is outsourcing located currently?  
Does the US dollar weakening help or hurt the company?  
What are significant announcements of products during fiscal year 2023?  
iPhone Net Sales?


---
# Disclaimer
Les informations contenues dans ce notebook sont fournies **à des fins purement académiques**, dans le cadre du cours d’**Intelligence Artificielle** dispensé par **Hugues Talbot** à **CentraleSupélec - MSTM**. Ce travail ne constitue en aucun cas un **conseil en investissement** ni une recommandation financière. Le contenu est destiné à un usage strictement pédagogique et **ne doit pas être diffusé, partagé ou publié** en dehors de ce contexte sans autorisation préalable.
###Yvan-Manuel BALEGUEL